In [2]:
import pandas as pd
from os import path, getcwd

In [20]:
base_dir = path.abspath(path.join(getcwd(), '..'))
df = pd.read_csv(path.join(base_dir, "tmp", "owid_covid_data.csv"), parse_dates=["date"])

In [21]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [19]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [29]:
df.shape

(429435, 67)

In [30]:
print("Total de linhas totalmente nulas: ", df.isnull().all().sum())

Total de linhas totalmente nulas:  0


In [31]:
df.isnull().sum().sort_values(ascending=False)

weekly_icu_admissions                   418442
weekly_icu_admissions_per_million       418442
excess_mortality                        416024
excess_mortality_cumulative_absolute    416024
excess_mortality_cumulative             416024
                                         ...  
total_cases_per_million                  17631
location                                     0
iso_code                                     0
date                                         0
population                                   0
Length: 67, dtype: int64

In [49]:
distinct_dim_table_location = df.loc[:, ["iso_code", "location", "continent"]].drop_duplicates()
distinct_dim_table_location[distinct_dim_table_location["continent"].isnull()]

distinct_dim_table_location_sec_ver = df.loc[:, ["iso_code", "location"]].drop_duplicates()

distinct_dim_table_location_iso_code = df.loc[:, ["iso_code"]].drop_duplicates()

In [57]:
print(distinct_dim_table_location.shape, distinct_dim_table_location_sec_ver.shape, distinct_dim_table_location_iso_code.shape)

(255, 3) (255, 2) (255, 1)


In [53]:
distinct_location_first = distinct_dim_table_location.isnull().sum()
distinct_location_second = distinct_dim_table_location_sec_ver.isnull().sum()
distinct_location_iso_code = distinct_dim_table_location_iso_code.isnull().sum()

In [54]:
distinct_location_first

iso_code      0
location      0
continent    12
dtype: int64

In [55]:
distinct_location_second

iso_code    0
location    0
dtype: int64

In [56]:
distinct_location_iso_code

iso_code    0
dtype: int64

In [63]:
full_match_iso_table = df[df["iso_code"].str.fullmatch(r"[A-Z]{3}")]["iso_code"].drop_duplicates()

In [64]:
full_match_iso_table.shape

(237,)

In [66]:
not_full_match_iso_table = df[~df["iso_code"].str.fullmatch(r"[A-Z]{3}")]["iso_code"].drop_duplicates()

In [67]:
not_full_match_iso_table

1674      OWID_AFR
20092     OWID_ASI
108832    OWID_ENG
118569    OWID_EUR
120253    OWID_EUN
163847    OWID_HIC
198670    OWID_KOS
217094    OWID_LIC
219818    OWID_LMC
277185    OWID_NAM
282207    OWID_CYN
282898    OWID_NIR
287618    OWID_OCE
337845    OWID_SCT
357564    OWID_SAM
406799    OWID_UMC
419856    OWID_WLS
422729    OWID_WRL
Name: iso_code, dtype: object

In [75]:
owid_codes = df.loc[df["iso_code"].str.startswith("OWID_"), "iso_code"].drop_duplicates().sort_values().to_list()

In [76]:
owid_codes

['OWID_AFR',
 'OWID_ASI',
 'OWID_CYN',
 'OWID_ENG',
 'OWID_EUN',
 'OWID_EUR',
 'OWID_HIC',
 'OWID_KOS',
 'OWID_LIC',
 'OWID_LMC',
 'OWID_NAM',
 'OWID_NIR',
 'OWID_OCE',
 'OWID_SAM',
 'OWID_SCT',
 'OWID_UMC',
 'OWID_WLS',
 'OWID_WRL']